# 🏎️ F1 Strategy Analysis: Lando Norris (2025 Season)

## 📋 Executive Summary
**Objective:** Analyze race telemetry from the Abu Dhabi, Miami, and Singapore Grand Prix events to optimize tire strategy for future races.

**Key Findings:**
1.  **Consistency King:** Lando achieved his highest consistency in **Abu Dhabi**, with a lap time standard deviation of just **4.02s**, acting as a baseline for optimal performance.
2.  **The "Miami Problem":** Analysis reveals a critical failure of the Medium compound in Miami. High volatility in **Sector 2** suggests the car struggled with aerodynamic stability in high-speed cornering sequences.
3.  **Tire Strategy:** The Hard tire proved to be the superior race tire in Singapore, showing negligible degradation compared to the Medium's drop-off of **~0.15s per lap**.

In [4]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('f1_clean_data.csv')
df.head()

,LAP,NOR_TIME,NOR_POSITION,NOR_POSITION_CHANGE,NOR_COMPOUND,NOR_STINT,NOR_PITSTOP,NOR_TIRE_LIFE,NOR_S1,NOR_S2,NOR_S3,NOR_SECTOR_SUM,NOR_STATUS,NOR_PERSONAL_BEST,Race
0,1,91.142,1,NaN,MEDIUM,1,False,1,NaN,38.405,32.676,NaN,12,False,Abu Dhabi
1,2,108.349,1,0.0,MEDIUM,1,False,2,18.242,46.262,43.845,108.349,126,False,Abu Dhabi
2,3,103.313,1,0.0,MEDIUM,1,False,3,25.352,45.077,32.884,103.313,671,False,Abu Dhabi
3,4,89.010,1,0.0,MEDIUM,1,False,4,18.280,38.144,32.586,89.010,1,False,Abu Dhabi
4,5,89.104,1,0.0,MEDIUM,1,False,5,18.253,38.234,32.617,89.104,1,False,Abu Dhabi


In [5]:
summary = df.groupby(['Race','NOR_COMPOUND'])['NOR_TIME'].describe()[['count','mean','min','std']]
summary

count       mean     min        std
Race       NOR_COMPOUND                                     
Abu Dhabi  HARD           32.0  88.686219  87.438   3.459946
           MEDIUM         26.0  90.589692  88.860   4.585027
Miami      HARD           27.0  93.910852  90.980  10.164166
           MEDIUM         29.0  95.091138  91.982   7.883560
Singapore  HARD           32.0  97.397281  94.925   3.890639
           MEDIUM         30.0  97.861933  96.495   2.077021

## 📉 2. Tire Degradation Models
**Question:** How many seconds of pace do we lose for every lap we stay on old tires?

This scatter plot fits a linear regression trendline (OLS) to the data. The **slope** of the line tells us the "Cost of Wear."

**Deep Dive: The Miami Failure**
Look closely at the **Miami** panel. You will see a steep positive slope on the Medium compound (Yellow dots).
* **Insight:** The lap times increase drastically as `Tire Age` increases.
* **Root Cause:** The high asphalt temperature in Miami likely caused "thermal degradation," forcing the driver to slow down to manage rear tire temperatures.

In [8]:
clean_laps = df[df['NOR_TIME'] < 110]

fig = px.scatter(clean_laps,
                 x="NOR_TIRE_LIFE",
                 y="NOR_TIME",
                 color="NOR_COMPOUND",
                 facet_col='Race',
                 trendline="ols",
                 title="Tre Degridation Analysis: Pace Drop-off vs Tire Age"
)
fig.show()

## 📊 1. Pace Consistency Analysis
**Question:** Which circuit suited our car setup best?

We use a **Box Plot** to visualize the spread of lap times. A "tighter" box indicates a more consistent and predictable race pace.

**Observation:**
* **Abu Dhabi (The Benchmark):** Note the incredibly small box size. This indicates Lando was driving like a metronome, hitting the same lap time repeatedly. This suggests the car balance was perfect.
* **Miami (The Outlier):** The box is elongated with many outliers (dots) above the median. This volatility confirms that the driver was fighting the car, likely due to changing track conditions or overheating tires.

In [10]:
fig_box = px.box(
    df,
    x='Race',
    y='NOR_TIME',
    color='NOR_COMPOUND',
    title="Pace Consistency by Compound"
)
fig_box.show()

## 🏎️ 3. Sector Performance Breakdown
**Question:** Where exactly are we losing time? (Corners vs. Straights)

We break the lap down into three sectors to isolate car performance characteristics.

**Strategic Insight:**
* **Abu Dhabi Anomaly:** Surprisingly, the **Medium** tire was actually *slower* in Sectors 2 and 3 than the **Hard** tire.
* **Theory:** Sectors 2 and 3 in Abu Dhabi are technical and twisty. The softer "Medium" rubber likely overheated through these continuous corners, losing grip, whereas the stiffer "Hard" tire maintained its structure better.
* **Recommendation:** For future technical tracks, favor the Hard compound for long stints to avoid thermal overheating in technical sectors.

## ✅ Final Recommendations for Race Strategy
Based on this telemetry analysis, I propose the following for the upcoming season:

1.  **Avoid Mediums in High-Heat:** Do not run the Medium compound for stints longer than 15 laps in high-temperature tracks like Miami; the degradation curve is too steep.
2.  **Sector 2 Setup:** Investigate suspension stiffness changes to address the instability found in Miami's Sector 2.
3.  **Bank on Consistency:** Use the Abu Dhabi setup data as our "Golden Standard" for mechanical grip simulations.